In [70]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

## 타이타닉 데이터 전처리 

In [71]:
titanic = sns.load_dataset('titanic')
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [72]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [73]:
titanic.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [74]:
# 불필요한 column 제거: class(pclass와 동일), who, adult_male, deck(결측값 많음), embarked_town(embarked와 동일), alive(survived와 동일)
drop_list = ['class', 'who', 'adult_male', 'deck', 'embark_town', 'alive']
titanic = titanic.drop(drop_list, axis=1)
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,alone
0,0,3,male,22.0,1,0,7.2500,S,False
1,1,1,female,38.0,1,0,71.2833,C,False
2,1,3,female,26.0,0,0,7.9250,S,True
3,1,1,female,35.0,1,0,53.1000,S,False
4,0,3,male,35.0,0,0,8.0500,S,True


In [75]:
# 중복된 값을 확인
titanic.duplicated().sum()

111

In [76]:
# 중복된 값은 첫 번째만 남기고 제거하여 inplace옵션으로 결과를 반영
titanic.drop_duplicates(keep='first', inplace=True)

In [77]:
# age의 결측값은 평균값으로 처리
titanic['age'].fillna(titanic['age'].mean(), inplace=True)
titanic['age'] = titanic['age'].astype(int)
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,alone
0,0,3,male,22,1,0,7.2500,S,False
1,1,1,female,38,1,0,71.2833,C,False
2,1,3,female,26,0,0,7.9250,S,True
3,1,1,female,35,1,0,53.1000,S,False
4,0,3,male,35,0,0,8.0500,S,True


In [78]:
# embarked는 최빈값으로 처리
titanic['embarked'].fillna(titanic['embarked'].value_counts().index[0], inplace=True)
titanic.isnull().sum()

survived    0
pclass      0
sex         0
age         0
sibsp       0
parch       0
fare        0
embarked    0
alone       0
dtype: int64

In [79]:
titanic.head(10)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,alone
0,0,3,male,22,1,0,7.2500,S,False
1,1,1,female,38,1,0,71.2833,C,False
2,1,3,female,26,0,0,7.9250,S,True
3,1,1,female,35,1,0,53.1000,S,False
4,0,3,male,35,0,0,8.0500,S,True
5,0,3,male,29,0,0,8.4583,Q,True
6,0,1,male,54,0,0,51.8625,S,True
7,0,3,male,2,3,1,21.0750,S,False
8,1,3,female,27,0,2,11.1333,S,False
9,1,2,female,14,1,0,30.0708,C,False


In [80]:
# 범주형 데이터를 수치형 데이터로 변환
titanic['sex'] = titanic['sex'].replace({'male': 0, 'female': 1})
titanic['embarked'] = titanic['embarked'].replace({'S':0, 'C':1, 'Q':2})
titanic['alone'] = titanic['alone'].replace({True:1,False:0}) 
titanic.head(10)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,alone
0,0,3,0,22,1,0,7.2500,0,0
1,1,1,1,38,1,0,71.2833,1,0
2,1,3,1,26,0,0,7.9250,0,1
3,1,1,1,35,1,0,53.1000,0,0
4,0,3,0,35,0,0,8.0500,0,1
5,0,3,0,29,0,0,8.4583,2,1
6,0,1,0,54,0,0,51.8625,0,1
7,0,3,0,2,3,1,21.0750,0,0
8,1,3,1,27,0,2,11.1333,0,0
9,1,2,1,14,1,0,30.0708,1,0


In [81]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
Index: 780 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   survived  780 non-null    int64  
 1   pclass    780 non-null    int64  
 2   sex       780 non-null    int64  
 3   age       780 non-null    int32  
 4   sibsp     780 non-null    int64  
 5   parch     780 non-null    int64  
 6   fare      780 non-null    float64
 7   embarked  780 non-null    int64  
 8   alone     780 non-null    int64  
dtypes: float64(1), int32(1), int64(7)
memory usage: 57.9 KB


## 교차 검증 

In [82]:
# 학습 데이터와 테스트 데이터로 분할하기 이전에 전처리한 titanic 데이터에서 label을 분리해준다
data = titanic.drop('survived', axis=1)
label = titanic['survived']

In [83]:
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=111)

In [84]:
model = DecisionTreeClassifier()
model_sp1 = DecisionTreeClassifier(random_state=111, max_depth=3)

#모델 학습
model.fit(X_train, y_train) 
model_sp1.fit(X_train, y_train) 

# X train 데이터로 예측 --> test 데이터로 예측
pred_train = model.predict(X_train) # train 예측
pred_test = model.predict(X_test) # test 예측

pred_train_sp = model_sp1.predict(X_train) # train 예측
pred_test_sp = model_sp1.predict(X_test) # test 예측

print("의사결정나무 예측 정확도 점수")
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test), '\n')

print("의사결정나무 w/ max_depth=3 예측 정확도 점수")
print(accuracy_score(y_train, pred_train_sp))
print(accuracy_score(y_test, pred_test_sp), '\n')

의사결정나무 예측 정확도 점수
0.9839743589743589
0.6794871794871795 

의사결정나무 w/ max_depth=3 예측 정확도 점수
0.7884615384615384
0.8205128205128205 



<일반 의사결정 나무 모델 vs max_depth=3인 의사결정 나무 모델>   
1. 일반적인 의사결정나무 모델을 학습하였을 때,, train 데이터에 대해 0.98에 가까운 예측도를 갖는다 --> 학습 데이터에 완벽하게 적합되었다(과적합)
2. max_depth를 3으로 한 의사결정 나무 모델을 학습하였을 때, train데이터에 대해 0.79의 예측도를 갖는다 --> 학습 데이터에 대해 일반적인 의사결정 나무보다 낮은 예측도
3. 학습되지 않은 test 데이터에 대한 예측 정확도는 비교적 과적합이 덜한 max_depth를 3으로 한 의사결정 나무 모델이 더 높았다 (0.82 > 0.68)

## Titanic Data의의 경우, KFold로 과적합 방지가 가능할까? 

In [85]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5) # n_splits: 데이터셋을 분할할 세트의 개수 --> 1세트만 test 데이터, 나머지는 train 데이터가 된다

cv_accuracy_train=[]    # train 데이터의 예측 정확도를 저장할 빈 리스트
cv_accuracy_test=[]     # test 데이터의 예측 정확도를 저장할 빈 리스트
kf_model = DecisionTreeClassifier(random_state=111, max_depth=3)

In [86]:
kfold

KFold(n_splits=5, random_state=None, shuffle=False)

In [87]:
n_iter = 0 # for문의 반복 횟수를 나타내며 초기값으로 0을 설정

for train_idx, test_idx in kfold.split(data):
    X_train, X_test = data.iloc[train_idx], data.iloc[test_idx]
    y_train, y_test = label.iloc[train_idx], label.iloc[test_idx]
    
    # 모델 학습
    kf_model.fit(X_train, y_train)
    
    # 예측
    kf_pred_train = kf_model.predict(X_train)
    df_pred_test = kf_model.predict(X_test)
    
    # 정확도를 계산하고 소수점 4자리로 반올림 
    accuracy_train = np.round(accuracy_score(y_train, kf_pred_train),4) 
    accuracy_test = np.round(accuracy_score(y_test, df_pred_test),4)
    # 해당하는 정확도 값을 해당 리스트에 append
    cv_accuracy_train.append(accuracy_train)
    cv_accuracy_test.append(accuracy_test)
    
    n_iter +=1 # 다음 시행을 위한 업데이트
    print('\n{} 번 train 교차 검증 정확도 :{} ,test 교차 검증 정확도 :{} '.format(n_iter,accuracy_train, accuracy_test))

print()
print('train 평균 정확도',round(np.mean(cv_accuracy_train), 4), 'test 평균 정확도',round(np.mean(cv_accuracy_test), 4))


1 번 train 교차 검증 정확도 :0.8045 ,test 교차 검증 정확도 :0.8013 

2 번 train 교차 검증 정확도 :0.8013 ,test 교차 검증 정확도 :0.7821 

3 번 train 교차 검증 정확도 :0.8077 ,test 교차 검증 정확도 :0.7628 

4 번 train 교차 검증 정확도 :0.8189 ,test 교차 검증 정확도 :0.7628 

5 번 train 교차 검증 정확도 :0.7869 ,test 교차 검증 정확도 :0.8077 

train 평균 정확도 0.8039 test 평균 정확도 0.7833


<KFold의 사용 유무에 따른 예측 정확도 차이 파악>
1. KFold를 사용한 의사결정 모델은 max_depth도 3으로 설정하였기에, 0.78(학습 예측), 0.82(테스트 예측)과 비교해야 한다
2. KFold를 사용한 의사결정 모델의 예측 정확도는 0.80(학습 에측), 0.78(테스트 예측)이 나왔다
3. 과적합 문제가 크게 개선되어 보이지 않고 거의 비슷한 수준의 예측 정확도가 관찰되었다. 

## Titanic Data의의 경우, SKFold로 과적합 방지가 가능할까? 

In [88]:
from sklearn.model_selection import StratifiedKFold

skf_titanic = StratifiedKFold(n_splits=5)      # skFold

skf_cv_accuracy_train = []                  # skfold를 통해 train 데이터로 예측한 정확도를 저장할 빈 리스트
skf_cv_accuracy_test = []                   # skfold를 통해 test 데이터로 예측한 정확도를 저장할 빈 리스트

skf_model = DecisionTreeClassifier(random_state=111, max_depth=3)   # depth가 3인 의사결정 나무 모델 설정

In [89]:
n_iter=0

for train_idx, test_idx in skf_titanic.split(data, label):     # skfold의 경우 split에 feature와 label이 둘 다 들어가야 한다
    X_train, X_test = data.iloc[train_idx], data.iloc[test_idx]
    y_train, y_test = label.iloc[train_idx], label.iloc[test_idx]
    
    # 학습
    skf_model.fit(X_train, y_train)
    # 예측
    skf_pred_train = skf_model.predict(X_train)
    skf_pred_test = skf_model.predict(X_test)
    
    # 정확도를 계산하고 소수점 4자리로 반올림 
    accuracy_train = np.round(accuracy_score(y_train, skf_pred_train),4)
    accuracy_test = np.round(accuracy_score(y_test, skf_pred_test),4)
    # 해당하는 정확도 값을 해당 리스트에 append
    skf_cv_accuracy_train.append(accuracy_train)
    skf_cv_accuracy_test.append(accuracy_test)
    
    n_iter +=1 # 다음 시행을 위한 업데이트
    print('\n{} 번 train 교차 검증 정확도 :{} ,test 교차 검증 정확도 :{} '.format(n_iter,accuracy_train, accuracy_test))
    
print()
print('train 평균 정확도',round(np.mean(skf_cv_accuracy_train), 4), 'test 평균 정확도',round(np.mean(skf_cv_accuracy_test), 4))


1 번 train 교차 검증 정확도 :0.8093 ,test 교차 검증 정확도 :0.6923 

2 번 train 교차 검증 정확도 :0.8125 ,test 교차 검증 정확도 :0.8013 

3 번 train 교차 검증 정확도 :0.8077 ,test 교차 검증 정확도 :0.7885 

4 번 train 교차 검증 정확도 :0.8205 ,test 교차 검증 정확도 :0.7564 

5 번 train 교차 검증 정확도 :0.7804 ,test 교차 검증 정확도 :0.8077 

train 평균 정확도 0.8061 test 평균 정확도 0.7692


<SKFold의 사용 유무에 따른 예측 정확도 차이 파악>
1. SKFold를 사용한 의사결정 모델도 max_depth도 3으로 설정하였으며 split도 5로 설정하여 기존 모델들과 다른 부분들은 일치시켰다
2. 비교 대상으로 Kfold를 쓰지 않은 경우(0.78(학습 예측), 0.82(테스트 예측)) Kfold를 쓴 경우(0.80(학습 에측), 0.78(테스트 예측))이 있다
3. SKFold를 사용한 의사결정 모델의 예측 정확도는 0.80(학습 에측), 0.77(테스트 예측)이 나왔다
4. SKFold를 사용한 경우도 과적합 문제가 크게 개선되어 보이지 않고 이전 모델들과 거의 비슷한 수준의 예측 정확도가 관찰되었다.